In [8]:
from os import getenv, getcwd
from borsdata_sdk import BorsdataAPI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.dpi'] = 200


api_key = '14e98818e51e42928578dd7e421f0e1d'
bors_api = BorsdataAPI(api_key)

In [13]:
'''report data = {'year': 2018, 'period': 5, 'revenues': 27772.0, 'gross_Income': 7049.0, 
'operating_Income': 1956.0, 'profit_Before_Tax': 1829.0, 'profit_To_Equity_Holders': 1322.0, 
'earnings_Per_Share': 5.2103, 'number_Of_Shares': 253.728, 'dividend': 1.85, 'intangible_Assets': 1999.0,
'tangible_Assets': 5468.0, 'financial_Assets': 91.0, 'non_Current_Assets': 7558.0, 'cash_And_Equivalents': 737.0, 
'current_Assets': 10477.0, 'total_Assets': 18035.0, 'total_Equity': 8996.0, 'non_Current_Liabilities': 3681.0, 
'current_Liabilities': 5358.0, 'total_Liabilities_And_Equity': 18035.0, 'net_Debt': 2667.0, 
'cash_Flow_From_Operating_Activities': 1090.0, 'cash_Flow_From_Investing_Activities': -723.0, 
'cash_Flow_From_Financing_Activities': -123.0, 'cash_Flow_For_The_Year': 244.0, 'free_Cash_Flow': 367.0, 
'stock_Price_Average': 134.95, 'stock_Price_High': 156.9, 'stock_Price_Low': 113.0, 
'report_Start_Date': '2018-01-01T00:00:00', 'report_End_Date': '2018-12-31T00:00:00', 
'broken_Fiscal_Year': False}'''

def load_report_data():
    instruments = bors_api.get_instruments()
    reports = {}
    idents = []
    tickers = []
    countrydict = {}
    for instrument in instruments:
        stock = instrument.ticker
        ident = instrument.insId
        reports[ident] = bors_api.get_instrument_reports(ident, 'year')
        idents.append(ident)
        tickers.append(stock)
        countrydict[instrument.insId] = instrument.countryId
    return idents, tickers, reports, countrydict

# This is the loading of the reporting data - only to be updated quarterly

In [14]:
'''This is not needed to be updated on a regular basis, only once in a while'''
idents, tickers, reports, countrydict = load_report_data()

In [15]:
print(reports[2][0].dividend)

1.85


In [17]:
#columns = ['id', 'ticker', 'close', 'dividend', 'earnings', 'growth_rev_5y', 'growth_ear_5y', 'growth_div_5y', 'equity_ration']
analysis_data = pd.DataFrame()


analysis_data['id'] = idents
analysis_data['ticker'] = tickers
analysis_data['close'] = None
analysis_data['dividend'] = None
analysis_data['earnings'] = None
analysis_data['growth_rev_5y'] = None
analysis_data['growth_ear_5y'] = None
analysis_data['growth_div_5y'] = None
analysis_data['equity_ratio'] = None
analysis_data['p/e'] = None
analysis_data['profit_return'] = None
analysis_data['direct_return'] = None
analysis_data['country'] = None

for i, row in analysis_data.iterrows():
    report = reports[int(row['id'])]
    
    if int(row['id']) in reports.keys() and len(reports[int(row['id'])])>0:
        index = row['id']
        row['country'] = int(countrydict[index])
        row['dividend'] = report[0].dividend
        row['earnings'] = report[0].earnings_Per_Share
        if len(report) > 4 and float(report[4].revenues)>0 and (float(report[0].revenues)-float(report[4].revenues))>0:
            row['growth_rev_5y'] = np.power(((float(report[0].revenues)-float(report[4].revenues))/float(report[4].revenues)),0.2)
        if len(report) > 4 and float(report[4].earnings_Per_Share)>0 and (float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))>0:
            row['growth_ear_5y'] = np.power(((float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))/float(report[4].earnings_Per_Share)),0.2)
        if len(report) > 4 and float(report[4].dividend)>0 and (float(report[0].dividend)-float(report[4].dividend))>0:
            row['growth_div_5y'] = np.power(((float(report[0].dividend)-float(report[4].dividend))/float(report[4].dividend)),0.2)
        if float(report[0].total_Assets)>0:
            row['equity_ratio']  = float(report[0].total_Equity)/float(report[0].total_Assets)
    analysis_data.iloc[i] = row

cwd = getcwd()
analysis_data.to_csv(cwd + '/analysis_data.csv')

# This segment are the buy candidates - to be updated daily

In [32]:
'''This should be updated daily'''

cwd = getcwd()
analysis_data = pd.read_csv(cwd + '/analysis_data.csv')
prices = bors_api.get_instrument_stock_price_last()
instruments = bors_api.get_instruments()
pricedict = {}


for price in prices:
    pricedict[price.i] = price.c 
    
for i, row in analysis_data.iterrows():
    index = row['id']
    row['close'] = pricedict[index]
    row['p/e'] = float(pricedict[index])/float(row['earnings']) if row['earnings'] is not None and float(row['earnings'])!=0 else None
    row['direct_return'] = float(row['dividend'])/float(pricedict[index]) if row['dividend'] is not None and float(row['dividend'])!=0 else None
    row['profit_return'] = 1/row['p/e'] if row['p/e'] is not None else None
    analysis_data.iloc[i] = row

In [33]:
n = 20
div_req = 0.02
prof_g = 0.1

filtered_results=analysis_data[(analysis_data.direct_return > div_req) 
                               & (analysis_data.profit_return>analysis_data.direct_return) 
                               & (analysis_data.growth_ear_5y > prof_g)
                               & (analysis_data.country == 1)].sort_values(['profit_return'], ascending=False)
tickers = filtered_results.ticker[0:n].to_dict()
print(filtered_results.head(n))

      Unnamed: 0    id    ticker   close  dividend  earnings  growth_rev_5y  \
1376        1376  1627    SAGA D   36.35    2.0000   10.0000       1.016123   
98            98   121    KLOV A   16.80    0.4600    3.9983       0.780245   
1368        1368  1619    KLOV B   16.96    0.4600    3.9983       0.780245   
276          276   349      TETY   74.30    8.0000   15.8981       0.790319   
43            43    57    CORE A   18.40    0.4500    3.2421       0.888574   
1357        1357  1608    CORE B   18.50    0.4500    3.2421       0.888574   
298          298   383      G5EN   85.80    2.5000   14.4504       1.474171   
22            22    32    BETS B   48.00    3.8900    7.7897       0.952911   
290          290   374      HEMF   93.25    2.4000   14.7532       0.892527   
239          239   300    GHUS B    4.25    0.1500    0.6619       0.939300   
177          177   217    SWED A  125.45   14.2000   19.0027       0.659888   
95            95   118      KAHL   19.54    2.0000  

In [34]:
filtered_results_div=analysis_data[(analysis_data.direct_return > div_req) 
                               & (analysis_data.profit_return>analysis_data.direct_return) 
                               & (analysis_data.growth_ear_5y > prof_g)
                               & (analysis_data.country == 1)].sort_values(['direct_return'], ascending=False)
tickers_div = filtered_results_div.ticker[0:n].to_dict()
print(filtered_results_div.head(n))

      Unnamed: 0    id    ticker   close  dividend  earnings  growth_rev_5y  \
131          131   159    NDA SE   59.37    7.1760    7.9127            NaN   
190          190   231  KIND SDB   51.98    5.9024    6.8927       1.123140   
177          177   217    SWED A  125.45   14.2000   19.0027       0.659888   
276          276   349      TETY   74.30    8.0000   15.8981       0.790319   
95            95   118      KAHL   19.54    2.0000    2.9159       0.324249   
203          203   245      DEDI   41.80    4.0000    4.2186       0.873185   
125          125   153     NET B   24.25    2.2500    2.4038       1.017817   
28            28    38      BORG   23.10    2.0000    2.3909       0.794419   
281          281   357      WISE   41.20    3.5000    4.7767       0.935060   
22            22    32    BETS B   48.00    3.8900    7.7897       0.952911   
1388        1388  1640    VOLV A  128.40   10.0000   12.2645       0.824616   
160          160   199     SEB A   83.54    6.5000  

In [35]:
#countries = bors_api.get_markets()
candidates = []
for i, result in tickers.items():
    if i in tickers_div.keys():
        candidates.append(i)
        print(result)
        print(instruments[i].__dict__)
        
print(analysis_data.iloc[candidates])

TETY
{'insId': 349, 'name': 'Tethys Oil', 'urlName': 'tethys-oil', 'instrument': 0, 'isin': 'SE0012455202', 'ticker': 'TETY', 'yahoo': 'TETY.ST', 'sectorId': 3, 'marketId': 2, 'branchId': 2, 'countryId': 1, 'listingDate': '2004-04-06T00:00:00'}
BETS B
{'insId': 32, 'name': 'Betsson', 'urlName': 'betsson', 'instrument': 0, 'isin': 'SE0012454379', 'ticker': 'BETS B', 'yahoo': 'BETS-B.ST', 'sectorId': 8, 'marketId': 1, 'branchId': 54, 'countryId': 1, 'listingDate': '1996-03-26T00:00:00'}
SWED A
{'insId': 217, 'name': 'Swedbank', 'urlName': 'swedbank', 'instrument': 0, 'isin': 'SE0000242455', 'ticker': 'SWED A', 'yahoo': 'SWED-A.ST', 'sectorId': 1, 'marketId': 1, 'branchId': 68, 'countryId': 1, 'listingDate': '1995-06-09T00:00:00'}
KAHL
{'insId': 118, 'name': 'KappAhl', 'urlName': 'kappahl', 'instrument': 0, 'isin': 'SE0010520981', 'ticker': 'KAHL', 'yahoo': 'KAHL.ST', 'sectorId': 8, 'marketId': 2, 'branchId': 43, 'countryId': 1, 'listingDate': '2006-02-23T00:00:00'}
RROS
{'insId': 191, 'n

# This segment are the potential sales candidates - to be updated daily

In [ ]:
filtered_results_neg=analysis_data[(analysis_data.direct_return > div_req) 
                               #& (analysis_data.profit_return>analysis_data.direct_return) 
                               & (analysis_data.growth_ear_5y > prof_g)
                               & (analysis_data.country == 1)].sort_values(['profit_return'], ascending=True)
tickers_neg = filtered_results_neg.ticker[0:n].to_dict()

for i, result in tickers_neg.items():
    print(i)
    print(instruments[i].name)
print(filtered_results_neg.head(n))